In [50]:
# import libraries
import tweepy as tp
import pprint
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from tweepy.streaming import StreamListener
import sys
import string
import time
from tweepy import Stream
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import sys,tweepy,csv,re
from textblob import TextBlob

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

from stop_words import get_stop_words

from textblob_fr import PatternTagger, PatternAnalyzer
from textblob_de import TextBlobDE

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, curdoc
from bokeh.models import ColumnDataSource, Select
from bokeh.models.glyphs import Line
from bokeh.models.widgets import Tabs, Panel
from bokeh.layouts import row

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms,modes
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet

import os
#import spacy
#from spacy.lang.en import English

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yassine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
class data:
    
    #Streams netflix and disney+ tweets in USA and Europ and stors it in a csv file
    def stream(self):
        
        geo_usa = "40.68908,-100.95860,1500km"
        geo_europe = "51.18348,10.03954,1200km"
        
        self.get_tweets("netflix", 140000, geo_usa, "europe", "en", "eng_usa")
        self.get_tweets("netflix", 140000, geo_europe, "europe", "en", "eng_europe")
        self.get_tweets("netflix", 140000, geo_europe, "europe", "fr", "fr_europe")
        self.get_tweets("netflix", 140000, geo_europe, "europe", "de", "ger_europe")
        self.get_tweets("netflix", 140000, geo_europe, "europe", "it", "it_europe")
        self.get_tweets("disneyplus", 140000, geo_usa, "europe", "en", "eng_usa")
        self.get_tweets("disneyplus", 140000, geo_europe, "europe", "en", "eng_europe")
        self.get_tweets("disneyplus", 140000, geo_europe, "europe", "fr", "fr_europe")
        self.get_tweets("disneyplus", 140000, geo_europe, "europe", "de", "ger_europe")
        self.get_tweets("disneyplus", 140000, geo_europe, "europe", "it", "it_europe")
    
    #Stream function
    def get_tweets(self, hashtag, count, geo, loc, lang, lang_loc):
        
        consumer_key = ''
        consumer_secret = ''
        access_token = ''
        access_token_secret = ''

        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        api = tweepy.API(auth,wait_on_rate_limit=True)	

        csvFile = open('df.csv', 'a', newline='')

        csvWriter = csv.writer(csvFile)

        for tweet in tweepy.Cursor(api.search,q="disney", count=count, lang=lang, since="2020-01-31", until="2020-02-09", geocode=geo).items():
            print (tweet.created_at, tweet.text)
            csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'), loc, hashtag, lang_loc])
    
    #import data from csv file
    def import_data(self):
        
        df = pd.read_csv("df.csv", names =["date", 'text', 'location', 'hashtag', 'lang_loc'])
        return df
    
    #preprocessing the dataframe 
    def data_preproc(self, df):
        
        df.lang_loc.fillna('eng_usa', inplace=True)
        df['date']=pd.to_datetime(df['date'])
        df=df.drop_duplicates(['text'])
        df.set_index('date', inplace=True)
        df=df.sort_values(by=['hashtag', 'date', 'location', 'lang_loc'])
        df = df.drop(df[df["lang_loc"]=="it_europe"].index)
        return df
    
    #preprocessing the tweets : removing stopwords and punctuation in each langage using text_process function bellow
    def new_text(self, df):
        
        new_text=[]
        
        for (tweet, lang_loc) in zip(df.text, df.lang_loc):
            
            if ((lang_loc == "eng_usa") | (lang_loc == "eng_europe")):
                new_tweet = self.text_process(tweet, "en")
                new_text.append(new_tweet)
                
            elif (lang_loc == "fr_europe"):
                new_tweet = self.text_process(tweet, "fr")
                new_text.append(new_tweet)
                
            elif (lang_loc == "it_europe"):
                new_tweet = self.text_process(tweet, "it")
                new_text.append(new_tweet)
                
            elif (lang_loc == "ger_europe"):
                new_tweet = self.text_process(tweet, "de")
                new_text.append(new_tweet)
                
        return new_text
    
    def text_process(self, mess, lang):
        
        stopwords=get_stop_words(lang)
        mess = [i for i in mess if i not in string.punctuation]
        mess = "".join(mess)
        mess = "".join(mess.split("b", 1))
        str1= " "
        return (str1.join([i for i in mess.split(" ") if (i.lower() not in stopwords)]))
    
    #calculate the polarity of each tweet in each langage
    def tweet_polarity(self, df):
        
        polarity = []
        for (tweet, lang) in zip(df.new_text, df["lang_loc"]):
            if ((lang == "eng_usa") | (lang == "eng_europe")):
                analysis = TextBlob(tweet)
                polarity.append(analysis.sentiment.polarity)
                
            elif (lang == "fr_europe"):
                analysis = TextBlob(tweet, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
                polarity.append(analysis.sentiment[0])
                
            elif (lang == "ger_europe"):
                analysis = TextBlobDE(tweet)
                polarity.append(analysis.sentiment.polarity)
                        
        return polarity 

Note : I've already streamed data in another notebook

# data preparation

In [19]:
data=data()

In [20]:
df=data.import_data()

In [21]:
df=data.data_preproc(df)

In [22]:
df["new_text"]=data.new_text(df)

In [23]:
polarity=data.tweet_polarity(df)

In [25]:
df["tweet_polarity"] = polarity

# Tweet encryption

In [26]:
key = os.urandom(32)

In [27]:
file = open('key.key', 'wb')  
file.write(key) 
file.close()

In [28]:
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())

In [29]:
aesEncryptor = cipher.encryptor()

In [42]:
tweets_encry=[]
for tweet in df["new_text"]:
    tweet_encry = tweet.encode("utf-16") + b"E" * (-len(tweet) % 16) 
    tweet_encry = aesEncryptor.update(tweet_encry)
    tweets_encry.append(tweet_encry)
    
df["text_encrypted"]=tweets_encry

In [47]:
df=df.drop(['text', 'new_text'], axis=1)

In [48]:
df.head()

,location,hashtag,lang_loc,tweet_polarity,text_encrypted
date,,,,,
2020-02-01 18:40:20,usa,disneyplus,eng_usa,0.100000,b'=\x18.w\x1e\xfc\xa3\x88\xbeD5b7\x95\xc8\xdd\...
2020-02-01 18:49:27,europe,disneyplus,fr_europe,0.400000,b'kt\xd2\x0e\x89UY\xe5\x1a.\xcc\xa3\xae\x1f.\x...
2020-02-01 18:53:38,usa,disneyplus,eng_usa,0.600000,"b'\xe2\xbe\x88G`g\x93""\xae\xfd_\x1b\xf0,\xbe5\..."
2020-02-01 18:56:18,europe,disneyplus,fr_europe,0.000000,"b""i\xb1q\xa09\xf1k\xbc\x81\x80.\x89(i\x87\xbfU..."
2020-02-01 18:58:49,usa,disneyplus,eng_usa,0.136364,b'\xc8\xdb\x18\xd4\xa2\x07XG\x8d\xe4\n1s\xec\x...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 131492 entries, 2020-02-01 18:40:20 to 2020-02-08 23:59:54
Data columns (total 6 columns):
text              131492 non-null object
location          131492 non-null object
hashtag           131492 non-null object
lang_loc          131492 non-null object
new_text          131492 non-null object
tweet_polarity    131492 non-null float64
dtypes: float64(1), object(5)
memory usage: 7.0+ MB


In [49]:
df.to_csv(r'C:\Users\Yassine\Desktop\Project tweets AIT JEDDI Yassine\df_prepared.csv')

The visualization is in the tweet_app file 